In [ ]:
%pylab inline

In [ ]:
import pandas as pd
import io
import requests
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
data = requests.get(url).content
df_wine = pd.read_csv(io.StringIO(data.decode('utf-8')))
columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
       'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids',
       'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
       'OD280/OD315 of diluted wines','Proline']
df_wine.columns = columns
df_wine.head()


In [ ]:
print('Class labels', np.unique(df_wine['Class label']))

In [ ]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
len(X_test), len(X_train)


In [ ]:
df_wine.iloc[:, 0].values

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5));
plt.subplot(1,2,1);
plt.plot(df_wine.Magnesium.values[:5], color='red', label='Magnesium')

plt.title('Without Effect of Normalization')
plt.xlabel('x values')
plt.ylabel('y values')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

In [ ]:
import matplotlib.pyplot as plt
mydf = pd.DataFrame(X_train_norm)

plt.figure(figsize=(15, 5));
plt.subplot(1,2,1);
plt.plot(mydf[5].values[:5], color='red', label='Magnesium')
plt.title('Effect of Normalization')
plt.xlabel('x values')
plt.ylabel('y values')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [ ]:
xx = np.arange(len(X_train_std))
yy1 = X_train_norm[:,1]
yy2 = X_train_std[:,1]
scatter(xx, yy1, color='b')
scatter(xx, yy2, color='r')

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l1', C=0.1) #L1 as paramenetr indicated L1 regularization
model.fit(X_train_std, y_train)
trainacc = model.score(X_train_std, y_train)
testacc = model.score(X_test_std, y_test)
print('Training accuracy:', trainacc)
print('Test accuracy:', testacc)

In [ ]:
model_input = df_wine.head(1)
model_input.drop('Class label', axis = 1, inplace= True)
output= model.predict(model_input)
print(output)

In [ ]:
import pandas as pd
import io
import requests
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
data = requests.get(url).content
df_wine = pd.read_csv(io.StringIO(data.decode('utf-8')))
columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
       'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids',
       'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
       'OD280/OD315 of diluted wines','Proline']

df_wine.columns = columns

In [ ]:
import sklearn.model_selection as model_selection
#split data for training, remove extras

wine_x = df_wine.drop(['Class label'], axis=1)
wine_y = df_wine['Class label']

wine_x_train, wine_x_test, wine_y_train, wine_y_test = model_selection.train_test_split(wine_x,
                                                                            wine_y,
                                                                            train_size=0.7,
                                                                            test_size=0.3,
                                                                            random_state=0)



In [ ]:
#### THIS IS WHERE WE PREINTIALIZE THE SCALAR ### ALSO NOTE WE ARE ASSUMING NO CATEGORICAL VALUE###
#### IF THERE ARE CATEGORICAL VALUES, YOU MAY USE ONEHOTENCODER or ORDINALENCODER etc ###
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
fitted_scaler = StandardScaler()
fitted_scaler = fitted_scaler.fit(wine_x_train)

In [ ]:
def pre_process(df,
                columns,
                fitted_scaler):
    ##Handle your preprocessing logic here, like identifying NaN, nulls etc
    ## scale the columns with the pre-built scaler
    _df = df.copy()
    _df = df[set(columns)].copy()
    _df = fitted_scaler.transform(_df)

    
    return _df

In [ ]:
wine_x_train_clean = pre_process(df = wine_x_train,
                                 columns = wine_x.columns,
                                 fitted_scaler = fitted_scaler)

wine_x_test_clean = pre_process(df = wine_x_test,
                                columns = wine_x.columns,
                                fitted_scaler = fitted_scaler)

In [ ]:
import mlflow.pyfunc
class wineModel(mlflow.pyfunc.PythonModel):
   
    ## defining objects needed for wineModel prediction. 
    def __init__(self,
                 model,
                 columns,
                 fitted_scaler,
                 pre_process):
        
        ## Setting up all needed objects
        self.model = model
        self.columns = columns
        self.fitted_scaler = fitted_scaler
        self.pre_process = pre_process
    
    ## define function with processing and feeding data into prediction at the end
    def predict(self,context,model_input):
        
       
        # run inputted dataset through our processing function
        # note: we are excluding the response columns here since not needed for deploy
        model_input_processed = self.pre_process(
                                   df = model_input,
                                   columns = self.columns,
                                   fitted_scaler = self.fitted_scaler)       
        
        # finally input the cleaned/adjusted dataset into our model for prediction
        obj = self.model.predict(model_input_processed)
        x = postprocess(obj)
        #return self.model.predict(model_input_processed)
        return x

In [ ]:
def postprocess(obj):
    result = obj.tostring()
    #WRITE WHATEVER YOU LIKE - FINDING THRESHOLD, SAVING TO DB ETC
    list = []  
    for x in obj :
        result = x 
        print(result)
        if result == 1:
            list.append('Class Label 1 Wine')
        elif result == 2:
            list.append('Class Label 2 Wine')
        else:
            list.append('Class Label 3 Wine')
    return numpy.array(list)   

In [ ]:
# define specific python and package versions for environment
mlflow_conda_env = {
 'name': 'mlflow-env',
 'channels': ['defaults'],
 'dependencies': ['python=3.6.2', {'pip': ['mlflow==1.10.0','scikit-learn','cloudpickle==1.5.0', 'numpy']}]
}

In [ ]:
import mlflow
import numpy
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ACCESS_KEY_ID=1B80PP32Z5VV6NJA6ABE
%env AWS_SECRET_ACCESS_KEY=fo0mwSE+MAWmNdhNJH0cY8qxXcj+sxbeKrZCk6Vl
%env MLFLOW_URL= http://localhost:5000


mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("ravion-Aug30-1")
with mlflow.start_run(run_name="Winde Model with Pre and Post Processing") as run:
            
    # log the performance metrics that we calculated earlier
    mlflow.log_metric("accuracy", trainacc)
    mlflow.log_metric("auc_score", testacc)
    
    # log model with all objects referenced in the leadsModel class
    mlflow.pyfunc.log_model(
        artifact_path = "wine_pyfunc",
        python_model = wineModel( model = model,
                                  columns = wine_x.columns,
                                  fitted_scaler = fitted_scaler,
                                  pre_process = pre_process
                                 ),
        conda_env = mlflow_conda_env
    )
    
    # save run_id and experiment_id for deployment
    run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    
    # end the mlflow run!
    mlflow.end_run()

In [ ]:
mlflow.end_run()

In [ ]:
import requests
def query_local_endpoint(input_json):
    response = requests.post('http://localhost:1234/invocations'
                           , headers = {'Content-Type': 'application/json'} 
                           , data=input_json)
    print(response)
    preds = response.json()
    return preds

In [ ]:
model_input = wine_x.head(1)
print(model_input)
output=query_local_endpoint(model_input.to_json(orient="split"))
print(output)

In [ ]:
import os
import sys
os.environ['PYSPARK_PYTHON']= "/Users/ravishankarnair/anaconda3/envs/py36/bin/python3"
os.environ['PYSPARK_DRIVER_PYTHON']= "/Users/ravishankarnair/anaconda3/envs/py36/bin/python3"
os.environ['PYLIB'] = os.environ['SPARK_HOME'] + "/python/lib"
sys.path.insert(0,os.environ['PYLIB'] + 'py4j-0.10.9-src.zip')
sys.path.insert(0,os.environ['PYLIB'] + 'pyspark.zip')
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .master("spark://ravion.local:7077") \
        .appName("OneClickBatchExecution") \
        .config("spark.executor.memory", "5G") \
        .config("spark.driver.memory", "5G") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .getOrCreate()
spark
 

In [ ]:
wine_x.columns

In [ ]:
from pyspark.sql.types import ArrayType, FloatType, StringType, StructField, StructType
from pyspark.sql.functions import udf, struct

# SYNTAX: spark_udf(spark, model_uri, result_type="<whatever type>") 
pyfunc_udf = mlflow.pyfunc.spark_udf(spark = spark, model_uri='s3://sample/19/1c47cf5cd059441baf78338a5dfce4fe/artifacts/wine_pyfunc', result_type=ArrayType(StringType()))

dataframe = spark.createDataFrame(wine_x)
#dataframe.columns
#dataframe.schema

#df = dataframe.withColumn("prediction", pyfunc_udf(struct('Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline')))
df = dataframe.withColumn("prediction", pyfunc_udf(struct(dataframe.columns)))


In [ ]:
df.head(1)

# import boto3
from botocore.client import Config


s3 = boto3.resource('s3',
                    endpoint_url='http://localhost:9000',
                    aws_access_key_id='1B80PP32Z5VV6NJA6ABE',
                    aws_secret_access_key='fo0mwSE+MAWmNdhNJH0cY8qxXcj+sxbeKrZCk6Vl',
                    config=Config(signature_version='s3v4'),
                    region_name='us-east-1')
for bucket in s3.buckets.all():
    print(bucket.name)

print(wine.col_names)

In [ ]:

loaded_model = mlflow.pyfunc.load_model('s3://sample/5/8e5241f88d6b4d7dab043a73a03d962b/artifacts/claims_pyfunc')

In [ ]:
import yaml

In [ ]:
yaml.safe_dump